##### *Copyright 2020 Google LLC*
*Licensed under the Apache License, Version 2.0 (the "License")*

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Fix top issues when converting a TF model for Edge TPU

This page shows how to fix some known issues when converting TensorFlow 2 models for the Edge TPU. 

<a href="https://colab.research.google.com/github/google-coral/tutorials/blob/master/fix_conversion_issues_ptq_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>
&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/google-coral/tutorials/blob/master/fix_conversion_issues_ptq_tf2.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View in GitHub"></a>


To run all the code in this tutorial, select **Runtime > Run all** in the Colab toolbar.

## Set up the environment

Import the Python libraries:

In [ ]:
import tensorflow as tf
assert float(tf.__version__[:3]) >= 2.3

import os
import numpy as np
import matplotlib.pyplot as plt

Install the Edge TPU Compiler:

In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler	

### Create quantization function

In [ ]:
def quantize_model(converter):
  # This generator provides a junk representative dataset
  # (It creates a poor model but is only for demo purposes)
  def representative_data_gen():
    for i in range(10):
      image = tf.random.uniform([1, 224, 224, 3])
      yield [image]
    
  converter.optimizations = [tf.lite.Optimize.DEFAULT]
  converter.representative_dataset = representative_data_gen
  converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
  converter.inference_input_type = tf.uint8
  converter.inference_output_type = tf.uint8

  return converter.convert()

## Can't compile due to dynamic batch size

The Edge TPU Compiler fails for some models such as MobileNetV1 if the input shape batch size is not set to 1, although this isn't exactly obvious from the compiler's output:

```
Invalid model: mobilenet_quant.tflite
Model not quantized
```




That error might be caused by something else, but you should try the following solution because although it's not required for all models, it shouldn't hurt.

### Solution for a Keras model object

In [ ]:
model = tf.keras.applications.MobileNet()

The following creates a TFLite file that will fail in the Edge TPU Compiler:

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = quantize_model(converter)

with open('mobilenet_quant_before.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
! edgetpu_compiler mobilenet_quant_before.tflite

It won't compile because the model has a dynamic batch size as shown here (`None` means it's dynamic):

In [ ]:
model.input.shape

So to fix it, we need to set that to 1:

In [ ]:
model.input.set_shape((1,) + model.input.shape[1:])
model.input.shape

Now we can convert it again and it will compile:

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = quantize_model(converter)

with open('mobilenet_quant_after.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
! edgetpu_compiler mobilenet_quant_after.tflite

### Solution for a SavedModel

If you're loading a SavedModel file, then the fix looks a little different.

So let's say you saved a model like this:

In [ ]:
model = tf.keras.applications.MobileNet()
save_path = os.path.join("mobilenet/1/")
tf.saved_model.save(model, save_path)

Ideally, you could later load the model like this:


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(save_path)

But the saved model's input still has a dynamic batch size, so you need to instead load the model with `saved_model.load()` and modify the input's concrete function so it has a batch size of 1. Then load it into `TFLiteConverter` using the concrete function:

In [ ]:
imported = tf.saved_model.load(save_path)
concrete_func = imported.signatures["serving_default"]
concrete_func.inputs[0].set_shape([1, 224, 224, 3])
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func], imported)

Now you can convert to TFLite and it will compile for the Edge TPU:

In [ ]:
tflite_model = quantize_model(converter)
with open('mobilenet_imported_quant.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
! edgetpu_compiler mobilenet_imported_quant.tflite

## Can't import a SavedModel without a signature

Sometimes, a SavedModel does not include a signature (such as when the model was built with a custom `tf.Module`), making it impossible to load using `TFLiteConverter`. So you can instead add the batch size as follows.

**Note:** If you created the model yourself, see how to [specify the signature during export](https://www.tensorflow.org/guide/saved_model#specifying_signatures_during_export) so this isn't a problem.

In [ ]:
# First get the Inception SavedModel, which is lacking a signature
!wget -O imagenet_inception_v2_classification_4.tar.gz https://tfhub.dev/google/imagenet/inception_v2/classification/4?tf-hub-format=compressed
!mkdir -p imagenet_inception_v2_classification_4
!tar -xvzf imagenet_inception_v2_classification_4.tar.gz --directory imagenet_inception_v2_classification_4

For example, this fails because the model has no signature:

In [ ]:
#converter = tf.lite.TFLiteConverter.from_saved_model("imagenet_inception_v2_classification_4")

Whereas other code above loads the input's concrete function by calling upon its "serving_default" signature, we can't do that if the model has no signature. So we instead get the concrete function by specifying its known input tensor shape:

In [ ]:
imported = tf.saved_model.load("imagenet_inception_v2_classification_4")
concrete_func = imported.__call__.get_concrete_function(
          tf.TensorSpec([1, 224, 224, 3]))
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func], imported)

Now we can convert and compile:

In [ ]:
tflite_model = quantize_model(converter)
with open('inceptionv2_imported_quant.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
! edgetpu_compiler inceptionv2_imported_quant.tflite